# Plan

- preprocess
    - 하나의 파일로 만들기
        - 데이터의 id는 (위도, 경도)
        - 각각 reserve와 store_info 합치기
        - 하나의 csv 파일로 합치기 
        - 요일, holiday flag column 추가
        - 식당 genre, area column 추가 
        - air_visit_data 이용해서 워크인 count

    - feature 찾기
        - one-hot encoding (요일, 식당 genre, area)
        - 분포 확인 (로그 변환 사용)
        - 상관도 검색

- train
    - train : val = 8 : 2
    - 여러 모델 테스트 (앙상블 사용 예상)
    - 하이퍼 파라미터 조정
    - RMSLE 값 기준

- test
    - 케글측에서 일부러 연휴 기간을 포함, 식당이 열지 않은 날도 있지만 최종 점수에 포함 안됨
    - submission 파일 복사 후 날짜 분리, 사용할 feature append
    - 예측
    - submission 파일에 예측값 append
    - 제출

In [3]:
import pandas as pd
import datetime as dt

In [4]:
# csv load
air_reserve_df = pd.read_csv('./data/air_reserve.csv')
hpg_reserve_df = pd.read_csv('./data/hpg_reserve.csv')
air_store_info_df = pd.read_csv('./data/air_store_info.csv')
hpg_store_info_df = pd.read_csv('./data/hpg_store_info.csv')
store_id_relation_df = pd.read_csv('./data/store_id_relation.csv')
date_info_df = pd.read_csv('./data/date_info.csv')
air_visit_data_df = pd.read_csv('./data/air_visit_data.csv')
sample_submission_df = pd.read_csv('./data/sample_submission.csv')

In [5]:
air_reserve_df.head()
# print(len(air_reserve_df['air_store_id'].unique()))
# print(len(air_reserve_df['air_store_id']))

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [6]:
air_store_info_df.head()
# print(len(air_store_info_df['air_store_id'].unique()))

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [7]:
hpg_reserve_df.head()
# print(len(hpg_reserve_df['hpg_store_id'].unique()))
# print(len(hpg_reserve_df))

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [8]:
hpg_store_info_df.head()
# print(len(hpg_store_info_df['hpg_store_id'].unique()))

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


In [9]:
store_id_relation_df.head()
# print(len(store_id_relation_df))

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


In [10]:
# store_id_relation.csv안에 id들이 1대1 맵핑인지 확인
check_air_id = store_id_relation_df.duplicated(['air_store_id'])
check_hpg_id = store_id_relation_df.duplicated(['hpg_store_id'])
for i, e in enumerate(check_air_id):
    if (e == True):
        print("air_id: " + i)
for i, e in enumerate(check_hpg_id):
    if (e == True):
        print("hpg_id: " + i)

print("Done") # 1대1 맵핑 확인

Done


In [11]:
date_info_df.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [12]:
air_visit_data_df.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [13]:
sample_submission_df.head()
# df = sample_submission_df.copy()
# df['new_id'] = df['id'].str.slice(start=0, stop=20)
# df.head()
# print(len(df['new_id'].unique()))

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


## preprocess

### 하나의 csv로 만들기

In [14]:
# air 예약정보와 가게정보 합치기
air_mixed_df = pd.merge(air_reserve_df, air_store_info_df)
# air_mixed_df.head()
# len(air_mixed_df)

In [15]:
hpg_mixed_df = pd.merge(hpg_reserve_df, hpg_store_info_df)
hpg_mixed_df.head()
# print(len(hpg_reserve_df))
# print(len(hpg_mixed_df))

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3,French,Hyōgo-ken Kōbe-shi None,34.692109,135.191698
1,hpg_dac72789163a3f47,2016-01-02 12:00:00,2016-01-01 20:00:00,2,French,Hyōgo-ken Kōbe-shi None,34.692109,135.191698
2,hpg_dac72789163a3f47,2016-01-03 19:00:00,2016-01-02 15:00:00,2,French,Hyōgo-ken Kōbe-shi None,34.692109,135.191698
3,hpg_dac72789163a3f47,2016-01-06 12:00:00,2016-01-06 08:00:00,2,French,Hyōgo-ken Kōbe-shi None,34.692109,135.191698
4,hpg_dac72789163a3f47,2016-01-10 17:00:00,2016-01-04 22:00:00,3,French,Hyōgo-ken Kōbe-shi None,34.692109,135.191698


In [16]:
# column name 공통되게 변경
air_mixed_new_columns_df = air_mixed_df.rename(columns = {"air_store_id":"id", "air_genre_name":"genre", "air_area_name": "area_name"}, inplace = False)
hpg_mixed_new_columns_df = hpg_mixed_df.rename(columns = {"hpg_store_id":"id", "hpg_genre_name":"genre", "hpg_area_name": "area_name"}, inplace = False)
mixed_df = pd.concat([air_mixed_new_columns_df, hpg_mixed_new_columns_df])
mixed_df

,id,visit_datetime,reserve_datetime,reserve_visitors,genre,area_name,latitude,longitude
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
1,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
2,air_877f79706adbfb06,2016-01-02 18:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
3,air_877f79706adbfb06,2016-01-02 21:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
4,air_877f79706adbfb06,2016-01-08 21:00:00,2016-01-04 20:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
...,...,...,...,...,...,...,...,...
871303,hpg_e106ddef2b137959,2017-04-24 18:00:00,2017-04-20 14:00:00,8,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619
871304,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-22 14:00:00,14,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619
871305,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-22 14:00:00,15,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619
871306,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-10 17:00:00,18,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619


In [25]:
# visit_datetime을 년월일로 변경
mixed_cp_df = mixed_df.copy()
mixed_cp_df['visit_date'] = mixed_cp_df['visit_datetime'].str.slice(start=0, stop=10)
# mixed_cp_df= mixed_cp_df.drop(['id', 'visit_datetime', 'reserve_datetime'], axis='columns')
mixed_cp_df

,id,visit_datetime,reserve_datetime,reserve_visitors,genre,area_name,latitude,longitude,visit_date
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-01
1,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-01
2,air_877f79706adbfb06,2016-01-02 18:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-02
3,air_877f79706adbfb06,2016-01-02 21:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-02
4,air_877f79706adbfb06,2016-01-08 21:00:00,2016-01-04 20:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2016-01-08
...,...,...,...,...,...,...,...,...,...
871303,hpg_e106ddef2b137959,2017-04-24 18:00:00,2017-04-20 14:00:00,8,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619,2017-04-24
871304,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-22 14:00:00,14,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619,2017-04-24
871305,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-22 14:00:00,15,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619,2017-04-24
871306,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-10 17:00:00,18,Japanese style,Tōkyō-to Chūō-ku None,35.682504,139.773619,2017-04-24


In [26]:
# 같은 날, 같은 식당의 reserve_visitors 합치기
merged_df = mixed_cp_df.groupby(by=['visit_date','latitude','longitude','genre','area_name', 'id'], as_index=False).sum()
merged_df

,visit_date,latitude,longitude,genre,area_name,id,reserve_visitors
0,2016-01-01,33.556881,130.415134,Seafood,Fukuoka-ken Fukuoka-shi None,hpg_a508a4e8b427279a,6
1,2016-01-01,33.576379,130.339313,Japanese style,Fukuoka-ken Fukuoka-shi Minamishō,hpg_b7f05c204af42b91,6
2,2016-01-01,33.576379,130.339313,Japanese style,Fukuoka-ken Fukuoka-shi Minamishō,hpg_f802d9079d5b1af3,2
3,2016-01-01,33.579734,130.401762,International cuisine,Fukuoka-ken Fukuoka-shi Ōmiya,hpg_962b3ed9f41e5b63,2
4,2016-01-01,33.586969,130.392801,Grilled meat,Fukuoka-ken Fukuoka-shi Daimyō,hpg_f6117f6d2533237f,2
...,...,...,...,...,...,...,...
590934,2017-05-31,35.682504,139.773619,Seafood,Tōkyō-to Chūō-ku None,hpg_77d300f2b1f22a8a,8
590935,2017-05-31,35.691384,139.701256,Spain Bar/Italian Bar,Tōkyō-to Shinjuku-ku None,hpg_ba19e6108767ebb6,4
590936,2017-05-31,35.691384,139.701256,Steak/Hamburger/Curry,Tōkyō-to Shinjuku-ku None,hpg_2bc4e1cbe5cd308b,1
590937,2017-05-31,35.695780,139.768453,International cuisine,Tōkyō-to Chiyoda-ku None,hpg_24bce7c3238bdd5c,40


In [27]:
# 요일과 holiday flag 추가하기
merged_with_day_df = pd.merge(merged_df, date_info_df, left_on='visit_date', right_on='calendar_date')
merged_with_day_df

,visit_date,latitude,longitude,genre,area_name,id,reserve_visitors,calendar_date,day_of_week,holiday_flg
0,2016-01-01,33.556881,130.415134,Seafood,Fukuoka-ken Fukuoka-shi None,hpg_a508a4e8b427279a,6,2016-01-01,Friday,1
1,2016-01-01,33.576379,130.339313,Japanese style,Fukuoka-ken Fukuoka-shi Minamishō,hpg_b7f05c204af42b91,6,2016-01-01,Friday,1
2,2016-01-01,33.576379,130.339313,Japanese style,Fukuoka-ken Fukuoka-shi Minamishō,hpg_f802d9079d5b1af3,2,2016-01-01,Friday,1
3,2016-01-01,33.579734,130.401762,International cuisine,Fukuoka-ken Fukuoka-shi Ōmiya,hpg_962b3ed9f41e5b63,2,2016-01-01,Friday,1
4,2016-01-01,33.586969,130.392801,Grilled meat,Fukuoka-ken Fukuoka-shi Daimyō,hpg_f6117f6d2533237f,2,2016-01-01,Friday,1
...,...,...,...,...,...,...,...,...,...,...
590934,2017-05-31,35.682504,139.773619,Seafood,Tōkyō-to Chūō-ku None,hpg_77d300f2b1f22a8a,8,2017-05-31,Wednesday,0
590935,2017-05-31,35.691384,139.701256,Spain Bar/Italian Bar,Tōkyō-to Shinjuku-ku None,hpg_ba19e6108767ebb6,4,2017-05-31,Wednesday,0
590936,2017-05-31,35.691384,139.701256,Steak/Hamburger/Curry,Tōkyō-to Shinjuku-ku None,hpg_2bc4e1cbe5cd308b,1,2017-05-31,Wednesday,0
590937,2017-05-31,35.695780,139.768453,International cuisine,Tōkyō-to Chiyoda-ku None,hpg_24bce7c3238bdd5c,40,2017-05-31,Wednesday,0


In [28]:
# genre 값 확인
merged_with_day_df['genre'].unique()

array(['Seafood', 'Japanese style', 'International cuisine',
       'Grilled meat', 'Italian', 'Dining bar', 'French',
       'Spain Bar/Italian Bar', 'Okonomiyaki/Monja/Teppanyaki',
       'Japanese cuisine/Kaiseki', 'Japanese food', 'Creation',
       'Japanese food in general', 'Shabu-shabu/Sukiyaki', 'Izakaya',
       'Chinese general', 'Creative Japanese food', 'Sushi',
       'Steak/Hamburger/Curry', 'Bistro', 'Udon/Soba', 'Italian/French',
       'Korean cuisine', 'Sweets', 'Karaoke', 'Bar/Cocktail',
       'Cafe/Sweets', 'Cafe', 'Thai/Vietnamese food', 'Pasta/Pizza',
       'Western food', 'Yakiniku/Korean food', 'Party', 'Amusement bar',
       'Cantonese food', 'Sichuan food', 'Shanghai food',
       'Spain/Mediterranean cuisine', 'Dim Sum/Dumplings',
       'Taiwanese/Hong Kong cuisine', 'Other', 'Creative cuisine',
       'Karaoke/Party', 'Asian'], dtype=object)

In [29]:
# area_name none 확인
merged_with_day_df['area_name'].unique()

array(['Fukuoka-ken Fukuoka-shi None',
       'Fukuoka-ken Fukuoka-shi Minamishō',
       'Fukuoka-ken Fukuoka-shi Ōmiya', 'Fukuoka-ken Fukuoka-shi Daimyō',
       'None None None', 'Fukuoka-ken Kitakyūshū-shi Sunatsu',
       'Hiroshima-ken Fukuyama-shi Sannomaruchō',
       'Ōsaka-fu Higashiōsaka-shi Wakaenishishinmachi',
       'Ōsaka-fu Ōsaka-shi Shinsaibashisuji',
       'Ōsaka-fu Ōsaka-shi Shinmachi', 'Hyōgo-ken Kōbe-shi None',
       'Ōsaka-fu Ōsaka-shi None', 'Ōsaka-fu Ōsaka-shi Nakanochō',
       'Osaka Prefecture Osaka None', 'Ōsaka-fu Suita-shi Enokichō',
       'Hyōgo-ken Kakogawa-shi Noguchichō Yoshino',
       'Ōsaka-fu Takatsuki-shi Kamitanabechō',
       'Shizuoka-ken Shizuoka-shi Hinodechō',
       'Shizuoka-ken Shizuoka-shi Irie',
       'Shizuoka-ken Numazu-shi Yanagisawa',
       'Tōkyō-to Machida-shi Nakamachi',
       'Tōkyō-to Shinagawa-ku Yutakachō', 'Tōkyō-to Tama-shi Tsurumaki',
       'Tōkyō-to Minato-ku Takanawa', 'Tōkyō-to Shibuya-ku Ebisunishi',
       'Tō

In [42]:
# air_visit_data 이용해서 워크인, 노쇼 등 확인
check_no_show_df = pd.merge(air_visit_data_df, merged_with_day_df, left_on=['air_store_id', 'visit_date'], right_on=['id','visit_date'])
check_no_show2_df = check_no_show_df.drop(['id', 'latitude', 'longitude', 'genre', 'area_name', 'calendar_date', 'day_of_week', 'holiday_flg'], axis='columns')
check_no_show2_df

,air_store_id,visit_date,visitors,reserve_visitors
0,air_35512c42db0868da,2016-02-27,6,12
1,air_ee3a01f0c71a769f,2016-01-04,61,2
2,air_ee3a01f0c71a769f,2016-01-08,21,3
3,air_ee3a01f0c71a769f,2016-01-09,57,25
4,air_ee3a01f0c71a769f,2016-01-10,32,5
...,...,...,...,...
28059,air_754ae581ad80cc9f,2017-04-14,3,2
28060,air_754ae581ad80cc9f,2017-04-15,39,20
28061,air_754ae581ad80cc9f,2017-04-19,6,6
28062,air_754ae581ad80cc9f,2017-04-22,47,28


In [43]:
# 노쇼와 워크인의 비중이 큰듯하여 예약에 관한 내용을 신경쓰지 않기로 함
temp_df = check_no_show_df.drop(['id', 'calendar_date'], axis='columns')
temp_df

,air_store_id,visit_date,visitors,latitude,longitude,genre,area_name,reserve_visitors,day_of_week,holiday_flg
0,air_35512c42db0868da,2016-02-27,6,35.717784,139.566260,Dining bar,Tōkyō-to Musashino-shi Midorichō,12,Saturday,0
1,air_ee3a01f0c71a769f,2016-01-04,61,34.710895,137.725940,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,2,Monday,0
2,air_ee3a01f0c71a769f,2016-01-08,21,34.710895,137.725940,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,3,Friday,0
3,air_ee3a01f0c71a769f,2016-01-09,57,34.710895,137.725940,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,25,Saturday,0
4,air_ee3a01f0c71a769f,2016-01-10,32,34.710895,137.725940,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,5,Sunday,0
...,...,...,...,...,...,...,...,...,...,...
28059,air_754ae581ad80cc9f,2017-04-14,3,37.874223,138.971940,Izakaya,Niigata-ken Niigata-shi Teraohigashi,2,Friday,0
28060,air_754ae581ad80cc9f,2017-04-15,39,37.874223,138.971940,Izakaya,Niigata-ken Niigata-shi Teraohigashi,20,Saturday,0
28061,air_754ae581ad80cc9f,2017-04-19,6,37.874223,138.971940,Izakaya,Niigata-ken Niigata-shi Teraohigashi,6,Wednesday,0
28062,air_754ae581ad80cc9f,2017-04-22,47,37.874223,138.971940,Izakaya,Niigata-ken Niigata-shi Teraohigashi,28,Saturday,0


In [48]:
# genre 값 확인
# temp_df['genre'].unique()
temp_df['genre'].value_counts(ascending=True)

Asian                              1
International cuisine              3
Karaoke/Party                     43
Other                            248
Creative cuisine                 331
Bar/Cocktail                     426
Okonomiyaki/Monja/Teppanyaki     932
Western food                     950
Cafe/Sweets                     1290
Yakiniku/Korean food            1508
Japanese food                   3076
Dining bar                      3406
Italian/French                  6071
Izakaya                         9779
Name: genre, dtype: int64

In [ ]:
# genre 통합


In [49]:
# area_name none 확인
# temp_df['area_name'].unique()
temp_df['area_name'].value_counts(ascending=True)

Tōkyō-to Musashino-shi Midorichō                  1
Tōkyō-to Taitō-ku Asakusa                         1
Hyōgo-ken Amagasaki-shi Higashinanamatsuchō       1
Tōkyō-to Setagaya-ku Okusawa                      1
Hyōgo-ken Kōbe-shi Motomachidōri                  1
                                               ... 
Shizuoka-ken Hamamatsu-shi Motoshirochō        1349
Hokkaidō Sapporo-shi Minami 3 Jōnishi          1421
Hiroshima-ken Hiroshima-shi Kokutaijimachi     1710
Fukuoka-ken Fukuoka-shi Daimyō                 1801
Tōkyō-to Shibuya-ku Shibuya                    2158
Name: area_name, Length: 71, dtype: int64

### Feature 찾기

## Train

## Test